# Interface Detection (NOTE: NOT WORKING)

Getting data:

In [1]:
import numpy as np
import pandas as pd
import pyrosetta
import re

import pyrosetta.rosetta.core.select.residue_selector as select
pyrosetta.init()

data = pd.read_csv("./raw_datasets/use_this_data.csv")
del data[data.columns[0]]

PyRosetta-4 2022 [Rosetta PyRosetta4.Release.python36.linux 2022.14+release.d95c9420af12eca758935f7e2b4d05be19e3a5fe 2022-04-07T08:56:27] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python36.linux r316 2022.14+release.d95c942 d95c9420af12eca758935f7e2b4d05be19e3a5fe http://www.pyrosetta.org 2022-04-07T08:56:27
core.init: command: PyRosetta -ex1 -ex2aro -database /projects/kasp8735/software/anaconda3/envs/cenv_flu2/lib/python3.6/site-packages/pyrosetta-2022.14+release.d95c942-py3.6-linux-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=1670943951 seed_offset=0 real_seed=1670943951
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=1670943951 RG_type=mt19937


Importing PDBs:

In [3]:
pose_list = []
print(data)
pdbs = data["#PDB"].unique()

for pdb in pdbs:
    print(pdb)
    pose = pyrosetta.rosetta.core.import_pose.get_pdb_and_cleanup(f"./PDBs/{pdb}.pdb")
    pose_list.append(pose)

print(len(pose_list))

     #PDB                    Mutations  ddG(kcal/mol)                Source  \
0    4FQY  H:S52I;H:S57T;H:I74K;H:F75S       5.031661  Phillips et al. 2021   
1    4FQY  H:N31S;H:S52I;H:I74K;H:F75S       5.031661  Phillips et al. 2021   
2    4FQY  H:S52I;H:T58A;H:I74K;H:F75S       5.031661  Phillips et al. 2021   
3    4FQY  H:N30S;H:S52I;H:I74K;H:F75S       5.031661  Phillips et al. 2021   
4    4FQY  H:N31S;H:S52I;H:S57T;H:I74K       5.031661  Phillips et al. 2021   
..    ...                          ...            ...                   ...   
819  4FQY                       H:N31S       1.394786  Phillips et al. 2021   
820  4FQY                       H:S57T       2.104520  Phillips et al. 2021   
821  4FQY                       H:F75S       2.796839  Phillips et al. 2021   
822  4FQY                       H:I74K       4.674663  Phillips et al. 2021   
823  4FQY                       H:S52I       4.843428  Phillips et al. 2021   

     LD  
0     4  
1     4  
2     4  
3     4  
4

Setting up selector:

In [4]:
neighbor_selector = select.NeighborhoodResidueSelector()
neighbor_selector.set_distance(8)

In [5]:
import re

clean_pose_list = []

for pose in pose_list:
    pose.pdb_info().name(re.sub(r"./PDBs/(\w{4}).pdb", r"\1", pose.pdb_info().name()))
    print(pose.pdb_info().name())

4FQY
3GBN
1DQJ
1MHP
1MLC
1N8Z
1VFB
1YY9
2NY7
2NYY
3BDY
3BE1
3BN9
3HFM
3NGB
1DVF
1KIQ
1KIP
1KIR
1NCA
1JRH
1NMB
3G6D
1XGU
1XGP
1XGQ
1XGR
1XGT
3N85
3L5X
1BJ1
1CZ8
2B2X
2NZ9
2BDN
5C6T
3SE8
1C08


Running Selector:

In [6]:
# Making it so the data can be .loc by pdb
interface_data = data.set_index("#PDB")

In [17]:
for pose in pose_list:
    pose.update_residue_neighbors()
    print(pose.pdb_info().name())
    for mutations in data[data['#PDB'] == pose.pdb_info().name()]["Mutations"]:
        mut_list = re.split('\n', mutations)
        for mutation in mut_list:
            res_selector = select.ResidueIndexSelector()
            muts = re.split(';', mutation)
            for mut in muts:
                print(mut)
                index, chain = re.split(":", re.sub(r"(\w):\w(\d+)\w", r"\2:\1", mut))
                res_selector.append_index(pose.pdb_info().pdb2pose(chain, int(index)))
            neighbor_selector.set_focus(res_selector.apply(pose))
            print(neighbor_selector.apply(pose))

4FQY
H:S52I
H:S57T
H:I74K
H:F75S
vector1_bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

RuntimeError: 

File: /home/benchmark/rosetta/source/src/core/select/residue_selector/ResidueIndexSelector.cc:189
[ ERROR ] UtilityExitException
ERROR: Error in core::select::residue_selector::ResidueIndexSelector::append_index(): The index must be greater than zero.

